# Player Ball By Ball data

### Making the final input.

Using all the dataframes we have already worked on, we are going to use them now to implement the final input.

**Inputs:**
- Player Bowling and Batting Stats
- Player lineup
- Ground names
- Innings Data
- Winner name

**Outputs:**
- Input dataframe that has ball by ball data of the following
    - Stats of current playing and those on bench.
    - Score uptil now
    - Total Wickets
    - Score this ball
    - Who is playing and who is on bench
    - The ground in one hot encoding
- An output dataframe

In [1]:
import numpy as np
import pandas as pd


**We are going to start by importing all the necessary files**

We are importing:
- The bowling stats
- The batting Stats
- The match data
- The innings data
- each match details




In [2]:
# Loading all the player stats data form disk
bowlingStats = pd.read_pickle("../Resources/playerData/bowling.pkl")
battingStats = pd.read_pickle("../Resources/playerData/batting.pkl")

# loading the match data from the disk
match_data = pd.read_pickle("../Resources/ground_player_lineup.pkl")
innings = pd.read_pickle("../Resources/match_data.pkl")
match_basics = pd.read_pickle("../Resources/file_index.pkl")
match_basics = match_basics[match_basics['Sex'] == 'male']

In [3]:
innings.index.levshape

(1773, 95, 20, 14)

In [4]:
innings.iloc[0].name[1]

'Austria'

### Stats of current playing and those on bench

**Some important points to note here:**
- We need data of both team, and select bowler and batsman accordingly.
- The proposed encoding would be as follows:

    | Position | Encoding |
    | -------- | -------- |
    | Player In | 1 |
    | Player Out | 0 |
    | Non Striker | 5 |
    | Striker | 10 |
    | Bowler | 10 |
    
- The Stats we will be gathering would be as follows:
      
    | Batsman | Bowler |
    | ------- | ------ |
    | Mat | Mat |
    | Inns | Inns |
    | NO | Overs |
    | Runs | Mdns |
    | HS | Runs |
    | Ave | Wkts |
    | BF | Ave |
    | SR | Econ |
    
    
  

**Some changes being done to data**

We will be changing the index of the data to make it simpler, besides that we will be dropping the nonneeded columns

In [5]:
bowlingStats = bowlingStats.set_index("Player")
battingStats = battingStats.set_index("Player")

In [6]:
battingStats.drop(['Country', 'Span', '100', '50', '0', '4s', '6s'], axis = 1, inplace = True)
bowlingStats.drop(['Country', 'Span', 'BBI', 'SR', '4', '5'], axis = 1, inplace = True)

In [7]:
bowlingStats

,Mat,Inns,Overs,Mdns,Runs,Wkts,Ave,Econ
Player,,,,,,,,
Shakib Al Hasan,117.000000,115.000000,422.300000,3.000000,2869.000000,140.000000,20.490000,6.790000
TG Southee,107.000000,105.000000,389.100000,3.000000,3179.000000,134.000000,23.720000,8.160000
Rashid Khan,82.000000,82.000000,312.200000,1.000000,1925.000000,130.000000,14.800000,6.160000
IS Sodhi,98.000000,95.000000,339.100000,0.000000,2701.000000,118.000000,22.880000,7.960000
SL Malinga,84.000000,83.000000,299.500000,1.000000,2225.000000,107.000000,20.790000,7.420000
...,...,...,...,...,...,...,...,...
B Zuiderent,6.000000,10.710079,32.795811,0.453754,241.972625,10.520946,28.038534,8.036798
Zulfiqar Ali Shah,1.000000,10.710079,32.795811,0.453754,241.972625,10.520946,28.038534,8.036798
Zulqarnain Haider,3.000000,10.710079,32.795811,0.453754,241.972625,10.520946,28.038534,8.036798


**Now as you can see in the match_data, I have to iterate over each of these to get a pair of series, one for the batting and one for the bowling team's batting.**

In [8]:
match_data

Details                \
                                       Ground Name   Winner Name   
Match Code                                                         
1381452        Sportpark Het Schootsveld, Deventer       Germany   
1381451        Sportpark Het Schootsveld, Deventer       Germany   
1381456             National Sports Academy, Sofia        Turkey   
1381457             National Sports Academy, Sofia        Serbia   
1381458             National Sports Academy, Sofia      Bulgaria   
...                                            ...           ...   
225271                               The Rose Bowl     Sri Lanka   
238195                       New Wanderers Stadium  South Africa   
226374      Brisbane Cricket Ground, Woolloongabba     Australia   
211028                               The Rose Bowl       England   
211048                                   Eden Park     Australia   

                                            Batting Team                 \
            Batting Team  Bowling Team                P1             P2   
Match Code                                                                
1381452          Austria       Germany  M Simpson-Parker  Iqbal Hossain   
1381451          Austria       Germany  M Simpson-Parker  Iqbal Hossain   
1381456          Croatia        Turkey          WK Bitis    Naseem Khan   
1381457          Croatia        Serbia          A Daxini    Naseem Khan   
1381458           Turkey      Bulgaria    Ilyas Ataullah         I Elec   
...                  ...           ...               ...            ...   
225271         Sri Lanka       England     ST Jayasuriya    WU Tharanga   
238195      South Africa     Australia          GC Smith      LE Bosman   
226374         Australia  South Africa          JR Hopes      DR Martyn   
211028           England     Australia    ME Trescothick       GO Jones   
211048         Australia   New Zealand      AC Gilchrist      MJ Clarke   

                                                                               \
                          P3            P4               P5                P6   
Match Code                                                                      
1381452      Razmal Shigiwal    A Randhawa     Shahil Momin       Mirza Ahsan   
1381451      Razmal Shigiwal    A Randhawa      Mirza Ahsan      Shahil Momin   
1381456            P Bosnjak  Sohail Ahmad         A Daxini           V Zanko   
1381457            P Bosnjak       V Zanko     A Magdalenic          WK Bitis   
1381458            A Turkmen        R Nath         M Ozturk  Muhammed Turkmen   
...                      ...           ...              ...               ...   
225271      DPMD Jayawardene    TM Dilshan    KC Sangakkara         RP Arnold   
238195              HH Gibbs    SM Pollock  JJ van der Wath    AB de Villiers   
226374            RT Ponting     A Symonds        MJ Clarke        MEK Hussey   
211028            A Flintoff  KP Pietersen       MP Vaughan    PD Collingwood   
211048             A Symonds    RT Ponting        DR Martyn         SM Katich   

            ...    Bowling Team                                    \
            ...              P2               P3               P4   
Match Code  ...                                                     
1381452     ...   J van Heerden    MJ Richardson  Faisal Mubashir   
1381451     ...   J van Heerden  VB Chikkannaiah  Faisal Mubashir   
1381456     ...  Ilyas Ataullah        A Turkmen           R Nath   
1381457     ...        A Dizija        LA Dunbar      MV Pavlovic   
1381458     ...       Zain Asif      Zaid Soulat         P Mishra   
...         ...             ...              ...              ...   
225271      ...      AJ Strauss     KP Pietersen   PD Collingwood   
238195      ...       SM Katich       RT Ponting        DR Martyn   
226374      ...    HH Dippenaar         HH Gibbs       MV Boucher   
211028      ...       ML Hayden        A Symonds        MJ Clarke   
211048 

#### Let's work on the index for each of these

In [9]:
bowlingStats.columns

Index(['Mat', 'Inns', 'Overs', 'Mdns', 'Runs', 'Wkts', 'Ave', 'Econ'], dtype='object')

In [10]:
bowlingIndex = pd.MultiIndex.from_product([["P" + str(i) for i in range(1, 12)], bowlingStats.columns])
battingIndex = pd.MultiIndex.from_product([["P" + str(i) for i in range(1, 12)], battingStats.columns])
print(bowlingIndex)
print(battingIndex)

MultiIndex([( 'P1',   'Mat'),
            ( 'P1',  'Inns'),
            ( 'P1', 'Overs'),
            ( 'P1',  'Mdns'),
            ( 'P1',  'Runs'),
            ( 'P1',  'Wkts'),
            ( 'P1',   'Ave'),
            ( 'P1',  'Econ'),
            ( 'P2',   'Mat'),
            ( 'P2',  'Inns'),
            ( 'P2', 'Overs'),
            ( 'P2',  'Mdns'),
            ( 'P2',  'Runs'),
            ( 'P2',  'Wkts'),
            ( 'P2',   'Ave'),
            ( 'P2',  'Econ'),
            ( 'P3',   'Mat'),
            ( 'P3',  'Inns'),
            ( 'P3', 'Overs'),
            ( 'P3',  'Mdns'),
            ( 'P3',  'Runs'),
            ( 'P3',  'Wkts'),
            ( 'P3',   'Ave'),
            ( 'P3',  'Econ'),
            ( 'P4',   'Mat'),
            ( 'P4',  'Inns'),
            ( 'P4', 'Overs'),
            ( 'P4',  'Mdns'),
            ( 'P4',  'Runs'),
            ( 'P4',  'Wkts'),
            ( 'P4',   'Ave'),
            ( 'P4',  'Econ'),
            ( 'P5',   'Mat'),
          

In [11]:
pd.DataFrame(columns = bowlingIndex)

Empty DataFrame
Columns: [(P1, Mat), (P1, Inns), (P1, Overs), (P1, Mdns), (P1, Runs), (P1, Wkts), (P1, Ave), (P1, Econ), (P2, Mat), (P2, Inns), (P2, Overs), (P2, Mdns), (P2, Runs), (P2, Wkts), (P2, Ave), (P2, Econ), (P3, Mat), (P3, Inns), (P3, Overs), (P3, Mdns), (P3, Runs), (P3, Wkts), (P3, Ave), (P3, Econ), (P4, Mat), (P4, Inns), (P4, Overs), (P4, Mdns), (P4, Runs), (P4, Wkts), (P4, Ave), (P4, Econ), (P5, Mat), (P5, Inns), (P5, Overs), (P5, Mdns), (P5, Runs), (P5, Wkts), (P5, Ave), (P5, Econ), (P6, Mat), (P6, Inns), (P6, Overs), (P6, Mdns), (P6, Runs), (P6, Wkts), (P6, Ave), (P6, Econ), (P7, Mat), (P7, Inns), (P7, Overs), (P7, Mdns), (P7, Runs), (P7, Wkts), (P7, Ave), (P7, Econ), (P8, Mat), (P8, Inns), (P8, Overs), (P8, Mdns), (P8, Runs), (P8, Wkts), (P8, Ave), (P8, Econ), (P9, Mat), (P9, Inns), (P9, Overs), (P9, Mdns), (P9, Runs), (P9, Wkts), (P9, Ave), (P9, Econ), (P10, Mat), (P10, Inns), (P10, Overs), (P10, Mdns), (P10, Runs), (P10, Wkts), (P10, Ave), (P10, Econ), (P11, Mat), (P11, Inns), (P11, Overs), (P11, Mdns), (P11, Runs), (P11, Wkts), (P11, Ave), (P11, Econ)]
Index: []

[0 rows x 88 columns]

#### Now it's time to make a function that returns the stats in a form of a series of each player

In [12]:
def return_bowling_stats(playerName):
    try:
        return bowlingStats.loc[playerName]
    except:
        return bowlingStats.loc['Average']
    
def return_batting_stats(playerName):
    try: 
        return battingStats.loc[playerName]
    except:
        return battingStats.loc['Average']

### Next, a function that takes in a match number and returns a dataframe made up of two series, batting team and bowling team turn

In [13]:
match_data.head()

Details                           \
                                    Ground Name Winner Name Batting Team   
Match Code                                                                 
1381452     Sportpark Het Schootsveld, Deventer     Germany      Austria   
1381451     Sportpark Het Schootsveld, Deventer     Germany      Austria   
1381456          National Sports Academy, Sofia      Turkey      Croatia   
1381457          National Sports Academy, Sofia      Serbia      Croatia   
1381458          National Sports Academy, Sofia    Bulgaria       Turkey   

                             Batting Team                                  \
           Bowling Team                P1             P2               P3   
Match Code                                                                  
1381452         Germany  M Simpson-Parker  Iqbal Hossain  Razmal Shigiwal   
1381451         Germany  M Simpson-Parker  Iqbal Hossain  Razmal Shigiwal   
1381456          Turkey          WK Bitis    Naseem Khan        P Bosnjak   
1381457          Serbia          A Daxini    Naseem Khan        P Bosnjak   
1381458        Bulgaria    Ilyas Ataullah         I Elec        A Turkmen   

                                                          ...    Bowling Team  \
                      P4            P5                P6  ...              P2   
Match Code                                                ...                   
1381452       A Randhawa  Shahil Momin       Mirza Ahsan  ...   J van Heerden   
1381451       A Randhawa   Mirza Ahsan      Shahil Momin  ...   J van Heerden   
1381456     Sohail Ahmad      A Daxini           V Zanko  ...  Ilyas Ataullah   
1381457          V Zanko  A Magdalenic          WK Bitis  ...        A Dizija   
1381458           R Nath      M Ozturk  Muhammed Turkmen  ...       Zain Asif   

                                                                             \
                         P3               P4                P5           P6   
Match Code                                                                    
1381452       MJ Richardson  Faisal Mubashir     Ghulam Ahmadi   Muslim Yar   
1381451     VB Chikkannaiah  Faisal Mubashir      Sahir Naqash  DA Blignaut   
1381456           A Turkmen           R Nath  Muhammed Turkmen     M Ozturk   
1381457           LA Dunbar      MV Pavlovic           S Tosic   V Zimonjic   
1381458         Zaid Soulat         P Mishra         Isa Zaroo    D Nikolov   

                                                                             \
                       P7                 P8               P9           P10   
Match Code                                                                    
1381452      SVE Bharathi       Abdul Bashir     Sachin Mandy  Sahir Naqash   
1381451     Ghulam Ahmadi       Sachin Mandy     SVE Bharathi    Muslim Yar   
1381456       Cihan Altun     Muhammet Bicer  Muhammet Kursat    Murat Ipek   
1381457        N Zimonjic  NR Johns-Wickberg     Rahman Ademi      AA Gajic   
1381458       Danyal Raja         A Ahmadhel  Gagandeep Singh   DV Varghese   

                           
                      P11  
Match Code                 
1381452           H Singh  
1381451           D Klein  
1381456      Murat Yilmaz  
1381457         M Sarenac  
1381458     Huzaif Yousuf  

[5 rows x 26 columns]

In [14]:
match_data.loc[1381452]['Batting Team']

P1     M Simpson-Parker
P2        Iqbal Hossain
P3      Razmal Shigiwal
P4           A Randhawa
P5         Shahil Momin
P6          Mirza Ahsan
P7           Aqib Iqbal
P8          Umair Tariq
P9        Jaweed Sadran
P10          MPS Cheema
P11          A Nathwani
Name: 1381452, dtype: object

**Function to return each innings Series i.e. (half of match)**

In [15]:
def returnMatchSeries(matchCode, firstTeam, secondTeam):
    batStats = match_data.loc[matchCode][firstTeam].apply(return_batting_stats).stack()
    bowlStats = match_data.loc[matchCode][secondTeam].apply(return_bowling_stats).stack()
    return pd.concat([batStats, bowlStats], keys = ['Batting Stats', 'Bowling Stats'])

**Function to return the whole match's dataframe**

In [16]:
def returnMatchDF(matchCode):
    try:
        battingTeam = match_data.loc[matchCode]['Details', 'Batting Team']
        bowlingTeam = match_data.loc[matchCode]['Details', 'Bowling Team']
        index = pd.MultiIndex.from_product([[matchCode], [battingTeam, bowlingTeam]])
        battingTeamInningsStats = returnMatchSeries(matchCode, 'Batting Team', 'Bowling Team')
        bowlingTeamInningsStats = returnMatchSeries(matchCode, 'Bowling Team', 'Batting Team')
        print("Doing this for match " + str(matchCode))
        return pd.DataFrame([battingTeamInningsStats, bowlingTeamInningsStats], index = index)
    except:
        print(str(matchCode) + " Failed")
        return

In [17]:
returnMatchDF(1381452)

Doing this for match 1381452


Batting Stats                                                \
                           P1                                                 
                          Mat  Inns   NO   Runs    HS    Ave     BF      SR   
1381452 Austria          33.0  26.0  2.0  428.0  65.0  17.83  440.0   97.27   
        Germany          40.0  34.0  7.0  460.0  61.0  17.03  449.0  102.44   

                             ... Bowling Stats                              \
                   P2        ...           P10             P11               
                  Mat  Inns  ...           Ave      Econ   Mat  Inns Overs   
1381452 Austria  25.0  21.0  ...     15.070000  7.400000  30.0   1.0   1.0   
        Germany  10.0   9.0  ...     28.038534  8.036798  29.0  28.0  91.0   

                                                    
                                                    
                Mdns   Runs  Wkts        Ave  Econ  
1381452 Austria  0.0   16.0   0.0  28.038534  16.0  
        Germany  0.0  628.0  34.0  18.470000   6.9  

[2 rows x 176 columns]

### Time to combine all the previous functions to get stats of every match

In [18]:
match_basics['Code']

8       1381452
9       1381451
10      1381456
11      1381457
12      1381458
         ...   
2696     237242
2697     226374
2698     222678
2699     211028
2700     211048
Name: Code, Length: 1836, dtype: int64

In [19]:
allData = match_basics['Code'].apply(returnMatchDF)

Doing this for match 1381452
Doing this for match 1381451
Doing this for match 1381456
Doing this for match 1381457
1381458 Failed
Doing this for match 1381453
1381454 Failed
1381455 Failed
1379592 Failed
1379585 Failed
Doing this for match 1379587
Doing this for match 1379579
1379581 Failed
Doing this for match 1379575
1379576 Failed
Doing this for match 1379584
1379590 Failed
1379582 Failed
Doing this for match 1379583
Doing this for match 1379586
1379589 Failed
Doing this for match 1379578
1379580 Failed
1380584 Failed
1380586 Failed
Doing this for match 1380587
1379577 Failed
1380583 Failed
1370791 Failed
Doing this for match 1370790
Doing this for match 1370785
Doing this for match 1370787
Doing this for match 1370782
Doing this for match 1370783
Doing this for match 1367731
1367732 Failed
1367728 Failed
Doing this for match 1367730
1367733 Failed
Doing this for match 1367727
1367729 Failed
1367725 Failed
1367726 Failed
1339618 Failed
1339617 Failed
Doing this for match 1339616
Do

Doing this for match 1276914
Doing this for match 1321293
Doing this for match 1321294
Doing this for match 1321295
1321296 Failed
Doing this for match 1276913
Doing this for match 1307477
1321289 Failed
1321290 Failed
1321291 Failed
Doing this for match 1321292
Doing this for match 1321286
Doing this for match 1321287
1321288 Failed
Doing this for match 1321281
1321282 Failed
1321283 Failed
1321284 Failed
Doing this for match 1303314
Doing this for match 1303313
Doing this for match 1321277
Doing this for match 1321278
1321279 Failed
1321280 Failed
Doing this for match 1303312
Doing this for match 1321273
Doing this for match 1321274
Doing this for match 1321275
1321276 Failed
1321481 Failed
1321482 Failed
Doing this for match 1321483
Doing this for match 1321484
Doing this for match 1321269
Doing this for match 1321270
Doing this for match 1321271
1321272 Failed
1321265 Failed
Doing this for match 1321266
Doing this for match 1321267
1321268 Failed
1321477 Failed
1321478 Failed
Doing

1282744 Failed
Doing this for match 1282745
Doing this for match 1283029
Doing this for match 1283030
Doing this for match 1283032
Doing this for match 1283089
Doing this for match 1273714
Doing this for match 1273715
1273712 Failed
Doing this for match 1273713
Doing this for match 1282742
1282743 Failed
Doing this for match 1283025
Doing this for match 1283026
Doing this for match 1283027
1283028 Failed
Doing this for match 1282740
1282741 Failed
Doing this for match 1283021
Doing this for match 1283022
Doing this for match 1283023
1283024 Failed
Doing this for match 1282738
1282739 Failed
Doing this for match 1280059
Doing this for match 1280062
Doing this for match 1280058
Doing this for match 1279385
1279386 Failed
Doing this for match 1280061
Doing this for match 1280063
1279383 Failed
Doing this for match 1280060
1279381 Failed
1279382 Failed
Doing this for match 1279379
1279380 Failed
1280057 Failed
Doing this for match 1273273
Doing this for match 1273272
1275051 Failed
Doing t

1199510 Failed
1199511 Failed
Doing this for match 1199503
Doing this for match 1199504
Doing this for match 1199505
Doing this for match 1199506
1199507 Failed
1199499 Failed
Doing this for match 1199500
Doing this for match 1199501
1199502 Failed
Doing this for match 1197528
1197529 Failed
Doing this for match 1197526
Doing this for match 1197527
Doing this for match 1198491
Doing this for match 1197524
Doing this for match 1197525
Doing this for match 1198490
1197522 Failed
Doing this for match 1197523
Doing this for match 1197521
Doing this for match 1198489
Doing this for match 1202011
1201685 Failed
Doing this for match 1202010
Doing this for match 1202009
Doing this for match 1201683
Doing this for match 1202008
1201682 Failed
Doing this for match 1202007
1201681 Failed
Doing this for match 1201680
Doing this for match 1187006
1197145 Failed
1197144 Failed
Doing this for match 1200428
Doing this for match 1187005
Doing this for match 1197143
Doing this for match 1200427
Doing th

951335 Failed
Doing this for match 951331
951333 Failed
Doing this for match 951329
Doing this for match 951325
951327 Failed
951321 Failed
Doing this for match 951323
951319 Failed
Doing this for match 951313
951315 Failed
Doing this for match 884351
Doing this for match 951309
951311 Failed
Doing this for match 951305
951307 Failed
Doing this for match 884349
Doing this for match 966765
Doing this for match 884347
966763 Failed
966761 Failed
966759 Failed
Doing this for match 966757
966755 Failed
Doing this for match 966753
966751 Failed
966749 Failed
966747 Failed
Doing this for match 966745
966741 Failed
966743 Failed
Doing this for match 800481
966739 Failed
966737 Failed
Doing this for match 800479
966713 Failed
966735 Failed
954743 Failed
954741 Failed
Doing this for match 963701
Doing this for match 963699
Doing this for match 954737
Doing this for match 963697
Doing this for match 954735
966373 Failed
967081 Failed
Doing this for match 895821
Doing this for match 953105
Doing 

412683 Failed
412679 Failed
412680 Failed
Doing this for match 412677
Doing this for match 412678
423788 Failed
Doing this for match 439139
Doing this for match 423787
Doing this for match 406198
Doing this for match 406197
440946 Failed
Doing this for match 440945
Doing this for match 439510
439511 Failed
439507 Failed
Doing this for match 439505
Doing this for match 439499
439495 Failed
Doing this for match 439497
Doing this for match 406207
Doing this for match 423782
Doing this for match 430885
Doing this for match 430884
Doing this for match 387564
Doing this for match 387563
426724 Failed
426723 Failed
Doing this for match 403386
Doing this for match 403385
350050 Failed
403375 Failed
401076 Failed
356017 Failed
Doing this for match 356016
356015 Failed
Doing this for match 356013
Doing this for match 356014
Doing this for match 356011
356012 Failed
Doing this for match 356009
Doing this for match 356010
Doing this for match 356007
356008 Failed
356005 Failed
Doing this for match

In [20]:
allData_list = allData.to_list()

In [21]:
match_player_stats = pd.concat(allData_list)
match_player_stats

Batting Stats                                            \
                                P1                                             
                               Mat  Inns    NO    Runs     HS    Ave      BF   
1381452 Austria               33.0  26.0   2.0   428.0   65.0  17.83   440.0   
        Germany               40.0  34.0   7.0   460.0   61.0  17.03   449.0   
1381451 Austria               33.0  26.0   2.0   428.0   65.0  17.83   440.0   
        Germany               40.0  34.0   7.0   460.0   61.0  17.03   449.0   
1381456 Croatia                9.0   7.0   1.0    93.0   32.0  15.50    72.0   
...                            ...   ...   ...     ...    ...    ...     ...   
226374  South Africa          33.0  33.0   2.0   982.0   89.0  31.67   770.0   
211028  England                3.0   3.0   0.0   166.0   72.0  55.33   131.0   
        Australia             13.0  13.0   1.0   272.0   48.0  22.66   192.0   
211048  Australia             13.0  13.0   1.0   272.0   48.0  22.66   192.0   
        New Zealand           71.0  70.0  10.0  2140.0  123.0  35.66  1571.0   

                                          ... Bowling Stats                  \
                                P2        ...           P10             P11   
                          SR   Mat  Inns  ...           Ave      Econ   Mat   
1381452 Austria        97.27  25.0  21.0  ...     15.070000  7.400000  30.0   
        Germany       102.44  10.0   9.0  ...     28.038534  8.036798  29.0   
1381451 Austria        97.27  25.0  21.0  ...     16.410000  6.200000  23.0   
        Germany       102.44  10.0   9.0  ...     18.470000  6.900000  20.0   
1381456 Croatia       129.16   8.0   5.0  ...     28.038534  3.000000   3.0   
...                      ...   ...   ...  ...           ...       ...   ...   
226374  South Africa  127.53   1.0   1.0  ...     18.230000  6.580000   2.0   
211028  England       126.71   2.0   2.0  ...     11.400000  8.140000   2.0   
        Australia     141.66   9.0   9.0  ...     16.330000  7.170000   2.0   
211048  Australia     141.66  34.0  28.0  ...     28.550000  8.210000   3.0   
        New Zealand   136.21   5.0   5.0  ...     11.400000  8.140000   2.0   

                                                                      
                                                                      
                      Inns Overs Mdns   Runs  Wkts        Ave   Econ  
1381452 Austria        1.0   1.0  0.0   16.0   0.0  28.038534  16.00  
        Germany       28.0  91.0  0.0  628.0  34.0  18.470000   6.90  
1381451 Austria       23.0  88.0  0.0  536.0  28.0  19.140000   6.09  
        Germany       20.0  50.3  1.0  415.0  13.0  31.920000   8.21  
1381456 Croatia        3.0   4.5  0.0   29.0   0.0  28.038534   6.00  
...                    ...   ...  ...    ...   ...        ...    ...  
226374  South Africa   2.0   7.3  0.0   49.0   4.0  12.250000   6.53  
211028  England        2.0   8.0  0.0   79.0   5.0  15.800000   9.87  
        Australia      2.0   6.3  0.0   42.0   1.0  42.000000   6.46  
211048  Australia      3.0  10.0  0.0   93.0   3.0  31.000000   9.30  
        New Zealand    2.0   8.0  0.0   79.0   5.0  15.800000   9.87  

[2526 rows x 176 columns]

### Saving this in a pickle file for later use

This is one of our main data that willl be fed directly to our neural network

In order to access it, you will use the following command:

```py
match_player_stats = pd.read_pickle("../Resources/MatchData/match_player_stats.pkl")
```

In [22]:
match_player_stats.to_pickle("../Resources/MatchData/match_player_stats.pkl")

In [23]:
match_player_stats = pd.read_pickle("../Resources/MatchData/match_player_stats.pkl")
match_player_stats

Batting Stats                                            \
                                P1                                             
                               Mat  Inns    NO    Runs     HS    Ave      BF   
1381452 Austria               33.0  26.0   2.0   428.0   65.0  17.83   440.0   
        Germany               40.0  34.0   7.0   460.0   61.0  17.03   449.0   
1381451 Austria               33.0  26.0   2.0   428.0   65.0  17.83   440.0   
        Germany               40.0  34.0   7.0   460.0   61.0  17.03   449.0   
1381456 Croatia                9.0   7.0   1.0    93.0   32.0  15.50    72.0   
...                            ...   ...   ...     ...    ...    ...     ...   
226374  South Africa          33.0  33.0   2.0   982.0   89.0  31.67   770.0   
211028  England                3.0   3.0   0.0   166.0   72.0  55.33   131.0   
        Australia             13.0  13.0   1.0   272.0   48.0  22.66   192.0   
211048  Australia             13.0  13.0   1.0   272.0   48.0  22.66   192.0   
        New Zealand           71.0  70.0  10.0  2140.0  123.0  35.66  1571.0   

                                          ... Bowling Stats                  \
                                P2        ...           P10             P11   
                          SR   Mat  Inns  ...           Ave      Econ   Mat   
1381452 Austria        97.27  25.0  21.0  ...     15.070000  7.400000  30.0   
        Germany       102.44  10.0   9.0  ...     28.038534  8.036798  29.0   
1381451 Austria        97.27  25.0  21.0  ...     16.410000  6.200000  23.0   
        Germany       102.44  10.0   9.0  ...     18.470000  6.900000  20.0   
1381456 Croatia       129.16   8.0   5.0  ...     28.038534  3.000000   3.0   
...                      ...   ...   ...  ...           ...       ...   ...   
226374  South Africa  127.53   1.0   1.0  ...     18.230000  6.580000   2.0   
211028  England       126.71   2.0   2.0  ...     11.400000  8.140000   2.0   
        Australia     141.66   9.0   9.0  ...     16.330000  7.170000   2.0   
211048  Australia     141.66  34.0  28.0  ...     28.550000  8.210000   3.0   
        New Zealand   136.21   5.0   5.0  ...     11.400000  8.140000   2.0   

                                                                      
                                                                      
                      Inns Overs Mdns   Runs  Wkts        Ave   Econ  
1381452 Austria        1.0   1.0  0.0   16.0   0.0  28.038534  16.00  
        Germany       28.0  91.0  0.0  628.0  34.0  18.470000   6.90  
1381451 Austria       23.0  88.0  0.0  536.0  28.0  19.140000   6.09  
        Germany       20.0  50.3  1.0  415.0  13.0  31.920000   8.21  
1381456 Croatia        3.0   4.5  0.0   29.0   0.0  28.038534   6.00  
...                    ...   ...  ...    ...   ...        ...    ...  
226374  South Africa   2.0   7.3  0.0   49.0   4.0  12.250000   6.53  
211028  England        2.0   8.0  0.0   79.0   5.0  15.800000   9.87  
        Australia      2.0   6.3  0.0   42.0   1.0  42.000000   6.46  
211048  Australia      3.0  10.0  0.0   93.0   3.0  31.000000   9.30  
        New Zealand    2.0   8.0  0.0   79.0   5.0  15.800000   9.87  

[2526 rows x 176 columns]

# Working on Player Status

- Like working on player stats for each ball, we will be working on player status now.
- Here is the player status key:
    | Status | Definition|
    | --- | --- |
    | 0 | Player is out |
    | 1 | Player is in game |
    | 5 | Player is non striker |
    | 10 | Player is baller | 
    | 10 | Player is batsman |

**Defining the function that returns a dataframe from each Country's data**

The countryDF is goin to be a default dataframe row that will be copied in the function to create each row like this

In [24]:
innings = innings.sort_index(level = "Match Code")
country_columns = pd.MultiIndex.from_product([['Batting Team', "Bowling Team"], ["P" + str(i) for i in range(1, 12)]])
countryDF = pd.DataFrame(columns = country_columns)
eachEntry = pd.Series(index = country_columns, dtype='float64')
countryDF = countryDF.append(eachEntry, ignore_index = True)
countryDF.fillna(1, inplace = True)
countryDF

Batting Team                              ... Bowling Team                 \
            P1 P2 P3 P4 P5 P6 P7 P8 P9 P10  ...           P2 P3 P4 P5 P6 P7   
0            1  1  1  1  1  1  1  1  1   1  ...            1  1  1  1  1  1   

                 
  P8 P9 P10 P11  
0  1  1   1   1  

[1 rows x 22 columns]

The eachBall and country_index here are being used to check if the function will work

In [25]:
country_index = 211028
eachBall = innings.loc[country_index].iloc[-3]
eachBall

batter               VS Solanki
bowler               GD McGrath
non_striker      PD Collingwood
batter runs                   0
extra runs                    0
total runs                    0
score                       175
extras                      NaN
wickets              VS Solanki
out                           1
Total Wickets                 7
non_boundary                NaN
review                      NaN
replacements                NaN
Name: (England, 19, 3), dtype: object

### Get Status Ball

- This function returns a dataframe row for each ball

In [26]:
# Get Status_ball will take in individual ball from the inning and return a dataframe with the status of that specific ball,
# This will be used with df.apply

def get_status_ball(eachBall, country_index):
    
    current_countryDF = countryDF.copy()
    # Working with the batter
    batsman = eachBall['batter']

    # This returns an index in form of multindex and player number is calculated from here
    batsmanIndex = match_data.columns[match_data.loc[country_index].isin([batsman])][0][1]
    current_countryDF['Batting Team', batsmanIndex] = 10

    # Working with the bowler
    bowler = eachBall['bowler']

    bowlerIndex = match_data.columns[match_data.loc[country_index].isin([bowler])][0][1]
    current_countryDF['Bowling Team', bowlerIndex] = 10

    # Working with the Non Striker
    non_striker = eachBall['non_striker']

    non_strikerIndex = match_data.columns[match_data.loc[country_index].isin([non_striker])][0][1]
    current_countryDF['Batting Team', non_strikerIndex] = 5

    # Working with the wickets
    wicket = eachBall['wickets']
    if not pd.isna(wicket):
        wicketIndex = match_data.columns[match_data.loc[country_index].isin([wicket])][0][1]
        current_countryDF['Batting Team', wicketIndex] = 0
        
    return current_countryDF

**This Shows how we are going to convert each innings wickets properly** 

**Along with a demo of how it will be used**


In [27]:
country_index = 211028
currcountry = innings.loc[country_index, 'Australia']

result = currcountry.apply(get_status_ball, args = (country_index,), axis = 1)
result = pd.concat(result.values, keys = result.index)

for column_name in result.columns:
    zero_index = result[result[column_name] == 0].index
    for index in zero_index:
        result.loc[index:, column_name] = 0
result

Batting Team                              ... Bowling Team            \
                 P1 P2 P3 P4 P5 P6 P7 P8 P9 P10  ...           P2  P3 P4 P5   
0  0 0           10  5  1  1  1  1  1  1  1   1  ...            1   1  1  1   
   1 0           10  5  1  1  1  1  1  1  1   1  ...            1   1  1  1   
   2 0           10  5  1  1  1  1  1  1  1   1  ...            1   1  1  1   
   3 0           10  5  1  1  1  1  1  1  1   1  ...            1   1  1  1   
   4 0           10  5  1  1  1  1  1  1  1   1  ...            1   1  1  1   
...             ... .. .. .. .. .. .. .. ..  ..  ...          ...  .. .. ..   
13 4 0            0  0  0  0  0  0  0  0  0   5  ...            1  10  1  1   
   5 0            0  0  0  0  0  0  0  0  0   5  ...            1  10  1  1   
14 0 0            0  0  0  0  0  0  0  0  0   5  ...            1   1  1  1   
   1 0            0  0  0  0  0  0  0  0  0   5  ...            1   1  1  1   
   2 0            0  0  0  0  0  0  0  0  0   5  ...            1   1  1  1   

                            
       P6 P7 P8 P9 P10 P11  
0  0 0  1  1  1  1  10   1  
   1 0  1  1  1  1  10   1  
   2 0  1  1  1  1  10   1  
   3 0  1  1  1  1  10   1  
   4 0  1  1  1  1  10   1  
...    .. .. .. ..  ..  ..  
13 4 0  1  1  1  1   1   1  
   5 0  1  1  1  1   1   1  
14 0 0  1  1  1  1   1  10  
   1 0  1  1  1  1   1  10  
   2 0  1  1  1  1   1  10  

[90 rows x 22 columns]

In [28]:
group = innings.loc[211028]
result = group.apply(get_status_ball, args = (country_index,), axis = 1)
result

Country    Over  Ball
Australia  0     0         Batting Team                              .....
                 1         Batting Team                              .....
                 2         Batting Team                              .....
                 3         Batting Team                              .....
                 4         Batting Team                              .....
                                               ...                        
England    19    1         Batting Team                               ....
                 2         Batting Team                               ....
                 3         Batting Team                              .....
                 4         Batting Team                               ....
                 5         Batting Team                              .....
Length: 215, dtype: object

In [29]:
eachCountry = pd.concat(result.values, keys = result.index)
eachCountry

Batting Team                                ... Bowling Team  \
                           P1 P2 P3 P4 P5  P6 P7  P8 P9 P10  ...           P2   
Australia 0  0 0           10  5  1  1  1   1  1   1  1   1  ...            1   
             1 0           10  5  1  1  1   1  1   1  1   1  ...            1   
             2 0           10  5  1  1  1   1  1   1  1   1  ...            1   
             3 0           10  5  1  1  1   1  1   1  1   1  ...            1   
             4 0           10  5  1  1  1   1  1   1  1   1  ...            1   
...                       ... .. .. .. ..  .. ..  .. ..  ..  ...          ...   
England   19 1 0            1  1  1  1  1   5  1  10  1   1  ...            1   
             2 0            1  1  1  1  1   5  1  10  1   1  ...            1   
             3 0            1  1  1  1  1   5  1   0  1   1  ...            1   
             4 0            1  1  1  1  1  10  1   1  5   1  ...            1   
             5 0            1  1  1  1  1   0  1   1  5   1  ...            1   

                                               
                 P3 P4 P5 P6 P7 P8 P9 P10 P11  
Australia 0  0 0  1  1  1  1  1  1  1  10   1  
             1 0  1  1  1  1  1  1  1  10   1  
             2 0  1  1  1  1  1  1  1  10   1  
             3 0  1  1  1  1  1  1  1  10   1  
             4 0  1  1  1  1  1  1  1  10   1  
...              .. .. .. .. .. .. ..  ..  ..  
England   19 1 0  1  1  1  1  1  1  1   1  10  
             2 0  1  1  1  1  1  1  1   1  10  
             3 0  1  1  1  1  1  1  1   1  10  
             4 0  1  1  1  1  1  1  1   1  10  
             5 0  1  1  1  1  1  1  1   1  10  

[215 rows x 22 columns]

In [30]:
eachCountry.index.names = ['Country', 'Over', 'Ball', 'todrop']
eachCountry = eachCountry.droplevel('todrop')
eachCountry

Batting Team                                ...  \
                              P1 P2 P3 P4 P5  P6 P7  P8 P9 P10  ...   
Country   Over Ball                                             ...   
Australia 0    0              10  5  1  1  1   1  1   1  1   1  ...   
               1              10  5  1  1  1   1  1   1  1   1  ...   
               2              10  5  1  1  1   1  1   1  1   1  ...   
               3              10  5  1  1  1   1  1   1  1   1  ...   
               4              10  5  1  1  1   1  1   1  1   1  ...   
...                          ... .. .. .. ..  .. ..  .. ..  ..  ...   
England   19   1               1  1  1  1  1   5  1  10  1   1  ...   
               2               1  1  1  1  1   5  1  10  1   1  ...   
               3               1  1  1  1  1   5  1   0  1   1  ...   
               4               1  1  1  1  1  10  1   1  5   1  ...   
               5               1  1  1  1  1   0  1   1  5   1  ...   

                    Bowling Team                               
                              P2 P3 P4 P5 P6 P7 P8 P9 P10 P11  
Country   Over Ball                                            
Australia 0    0               1  1  1  1  1  1  1  1  10   1  
               1               1  1  1  1  1  1  1  1  10   1  
               2               1  1  1  1  1  1  1  1  10   1  
               3               1  1  1  1  1  1  1  1  10   1  
               4               1  1  1  1  1  1  1  1  10   1  
...                          ... .. .. .. .. .. .. ..  ..  ..  
England   19   1               1  1  1  1  1  1  1  1   1  10  
               2               1  1  1  1  1  1  1  1   1  10  
               3               1  1  1  1  1  1  1  1   1  10  
               4               1  1  1  1  1  1  1  1   1  10  
               5               1  1  1  1  1  1  1  1   1  10  

[215 rows x 22 columns]

In [31]:
# get Status_group with take in individual group from the innings and return a dataframe with a status of that specific country
# This will be used with df.apply

def get_status_group(group):
    
    # Get Status_ball will take in individual ball from the inning and return a dataframe with the status of that specific ball,
    # This will be used with df.apply
    try:
        country_index = group.name[0]
        result = group.apply(get_status_ball, args = (country_index,), axis = 1)
        result = pd.concat(result.values, keys = result.index)
        result.index.names = ['Match Code', 'Country', 'Over', 'Ball', 'todrop']
        result = result.droplevel(['todrop', 'Match Code', 'Country'])
        for column_name in result.columns:
            zero_index = result[result[column_name] == 0].index
            for index in zero_index:
                result.loc[index:, column_name] = 0
    except:
        return
    else:
        return result
        

In [32]:
check_innings = innings.iloc[0:500]
check_matches = check_innings.groupby(['Match Code', 'Country'])
check_matches.apply(get_status_group)

Batting Team                               \
                                           P1 P2 P3 P4 P5 P6 P7 P8  P9 P10   
Match Code Country     Over Ball                                             
211028     Australia   0    0              10  5  1  1  1  1  1  1   1   1   
                            1              10  5  1  1  1  1  1  1   1   1   
                            2              10  5  1  1  1  1  1  1   1   1   
                            3              10  5  1  1  1  1  1  1   1   1   
                            4              10  5  1  1  1  1  1  1   1   1   
...                                       ... .. .. .. .. .. .. ..  ..  ..   
211048     New Zealand 19   1               0  0  0  0  0  0  0  0  10   5   
                            2               0  0  0  0  0  0  0  0   0   5   
                            3               0  0  0  0  0  0  0  0   0   5   
                            4               0  0  0  0  0  0  0  0   0   5   
                            5               0  0  0  0  0  0  0  0   0   0   

                                  ... Bowling Team                           \
                                  ...           P2 P3 P4 P5 P6 P7 P8 P9 P10   
Match Code Country     Over Ball  ...                                         
211028     Australia   0    0     ...            1  1  1  1  1  1  1  1  10   
                            1     ...            1  1  1  1  1  1  1  1  10   
                            2     ...            1  1  1  1  1  1  1  1  10   
                            3     ...            1  1  1  1  1  1  1  1  10   
                            4     ...            1  1  1  1  1  1  1  1  10   
...                               ...          ... .. .. .. .. .. .. ..  ..   
211048     New Zealand 19   1     ...            1  1  1  1  1  1  1  1   1   
                            2     ...            1  1  1  1  1  1  1  1   1   
                            3     ...            1  1  1  1  1  1  1  1   1   
                            4     ...            1  1  1  1  1  1  1  1   1   
                            5     ...            1  1  1  1  1  1  1  1   1   

                                      
                                 P11  
Match Code Country     Over Ball      
211028     Australia   0    0      1  
                            1      1  
                            2      1  
                            3      1  
                            4      1  
...                               ..  
211048     New Zealand 19   1     10  
                            2     10  
                            3     10  
                            4     10  
                            5     10  

[460 rows x 22 columns]

In [33]:
matches = innings.groupby(['Match Code', 'Country'])
player_status = matches.apply(get_status_group)

In [34]:
player_status

Batting Team                               ...  \
                                         P1 P2 P3 P4 P5 P6 P7 P8  P9 P10  ...   
Match Code Country   Over Ball                                            ...   
211028     Australia 0    0              10  5  1  1  1  1  1  1   1   1  ...   
                          1              10  5  1  1  1  1  1  1   1   1  ...   
                          2              10  5  1  1  1  1  1  1   1   1  ...   
                          3              10  5  1  1  1  1  1  1   1   1  ...   
                          4              10  5  1  1  1  1  1  1   1   1  ...   
...                                     ... .. .. .. .. .. .. ..  ..  ..  ...   
1381458    Turkey    19   1               0  0  0  0  0  0  0  0  10   5  ...   
                          2               0  0  0  0  0  0  0  0  10   5  ...   
                          3               0  0  0  0  0  0  0  0  10   5  ...   
                          4               0  0  0  0  0  0  0  0  10   0  ...   
                          5               0  0  0  0  0  0  0  0   5   0  ...   

                               Bowling Team                               
                                         P2 P3 P4 P5 P6 P7 P8 P9 P10 P11  
Match Code Country   Over Ball                                            
211028     Australia 0    0               1  1  1  1  1  1  1  1  10   1  
                          1               1  1  1  1  1  1  1  1  10   1  
                          2               1  1  1  1  1  1  1  1  10   1  
                          3               1  1  1  1  1  1  1  1  10   1  
                          4               1  1  1  1  1  1  1  1  10   1  
...                                     ... .. .. .. .. .. .. ..  ..  ..  
1381458    Turkey    19   1               1  1  1  1  1  1  1  1   1  10  
                          2               1  1  1  1  1  1  1  1   1  10  
                          3               1  1  1  1  1  1  1  1   1  10  
                          4               1  1  1  1  1  1  1  1   1  10  
                          5               1  1  1  1  1  1  1  1   1  10  

[401800 rows x 22 columns]

## Saving to disk

To access this, use the code:

```py
player_status = pd.read_pickle("../Resources/MatchData/player_status.pkl")
player_status
```

In [35]:
player_status.to_pickle("../Resources/MatchData/player_status.pkl")

In [36]:
player_status = pd.read_pickle("../Resources/MatchData/player_status.pkl")
player_status

Batting Team                               ...  \
                                         P1 P2 P3 P4 P5 P6 P7 P8  P9 P10  ...   
Match Code Country   Over Ball                                            ...   
211028     Australia 0    0              10  5  1  1  1  1  1  1   1   1  ...   
                          1              10  5  1  1  1  1  1  1   1   1  ...   
                          2              10  5  1  1  1  1  1  1   1   1  ...   
                          3              10  5  1  1  1  1  1  1   1   1  ...   
                          4              10  5  1  1  1  1  1  1   1   1  ...   
...                                     ... .. .. .. .. .. .. ..  ..  ..  ...   
1381458    Turkey    19   1               0  0  0  0  0  0  0  0  10   5  ...   
                          2               0  0  0  0  0  0  0  0  10   5  ...   
                          3               0  0  0  0  0  0  0  0  10   5  ...   
                          4               0  0  0  0  0  0  0  0  10   0  ...   
                          5               0  0  0  0  0  0  0  0   5   0  ...   

                               Bowling Team                               
                                         P2 P3 P4 P5 P6 P7 P8 P9 P10 P11  
Match Code Country   Over Ball                                            
211028     Australia 0    0               1  1  1  1  1  1  1  1  10   1  
                          1               1  1  1  1  1  1  1  1  10   1  
                          2               1  1  1  1  1  1  1  1  10   1  
                          3               1  1  1  1  1  1  1  1  10   1  
                          4               1  1  1  1  1  1  1  1  10   1  
...                                     ... .. .. .. .. .. .. ..  ..  ..  
1381458    Turkey    19   1               1  1  1  1  1  1  1  1   1  10  
                          2               1  1  1  1  1  1  1  1   1  10  
                          3               1  1  1  1  1  1  1  1   1  10  
                          4               1  1  1  1  1  1  1  1   1  10  
                          5               1  1  1  1  1  1  1  1   1  10  

[401800 rows x 22 columns]

In [37]:
innings

batter         bowler      non_striker  \
Match Code Country   Over Ball                                                 
211028     Australia 0    0     AC Gilchrist        D Gough        ML Hayden   
                          1     AC Gilchrist        D Gough        ML Hayden   
                          2     AC Gilchrist        D Gough        ML Hayden   
                          3     AC Gilchrist        D Gough        ML Hayden   
                          4     AC Gilchrist        D Gough        ML Hayden   
...                                      ...            ...              ...   
1381458    Turkey    19   1     Murat Yilmaz  Huzaif Yousuf  Muhammet Kursat   
                          2     Murat Yilmaz  Huzaif Yousuf  Muhammet Kursat   
                          3     Murat Yilmaz  Huzaif Yousuf  Muhammet Kursat   
                          4     Murat Yilmaz  Huzaif Yousuf  Muhammet Kursat   
                          5       Murat Ipek  Huzaif Yousuf     Murat Yilmaz   

                                batter runs  extra runs  total runs  score  \
Match Code Country   Over Ball                                               
211028     Australia 0    0               0           0           0      0   
                          1               4           0           4      4   
                          2               0           0           0      4   
                          3               0           0           0      4   
                          4               4           0           4      8   
...                                     ...         ...         ...    ...   
1381458    Turkey    19   1               0           0           0    146   
                          2               0           0           0    146   
                          3               0           0           0    146   
                          4               0           1           1    147   
                          5               0           0           0    147   

                                      extras          wickets  out  \
Match Code Country   Over Ball                                       
211028     Australia 0    0              NaN              NaN    0   
                          1              NaN              NaN    0   
                          2              NaN              NaN    0   
                          3              NaN              NaN    0   
                          4              NaN              NaN    0   
...                                      ...              ...  ...   
1381458    Turkey    19   1              NaN              NaN    0   
                          2              NaN              NaN    0   
                          3              NaN              NaN    0   
                          4     {'wides': 1}  Muhammet Kursat    1   
                          5              NaN       Murat Ipek    1   

                                Total Wickets non_boundary review replacements  
Match Code Country   Over Ball                                                  
211028     Australia 0    0                 0          NaN    NaN          NaN  
                          1                 0          NaN    NaN          NaN  
                          2                 0          NaN    NaN          NaN  
                          3                 0          NaN    NaN          NaN  
                          4                 0          NaN    NaN          NaN  
...                                       ...          ...    ...          ...  
1381458    Turkey    19   1                 8          NaN    NaN          NaN  
                          2                 8          NaN    NaN          NaN  
                          3                 8          NaN    NaN          NaN  
                          4                 9          NaN    NaN          NaN  
                          5                10          NaN    NaN          NaN  

[407789 rows x 14